# 実習8

#### GANを体験する

以下のHPを参考にして下さい。
https://elix-tech.github.io/ja/2017/02/06/gan.html
https://qiita.com/God_KonaBanana/items/293d49e3c34601a1810b

In [1]:
# generatorを作成

from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Convolution2D

def generator_model():
    model = Sequential()
    model.add(Dense(input_dim=100, units=1024)) #100次元のノイズベクトルがinput
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(128*7*7))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Reshape((7, 7, 128), input_shape=(128*7*7,))) #サイズ7x7　128chの画像
    model.add(UpSampling2D((2, 2)))
    model.add(Convolution2D(64, (5,5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(UpSampling2D((2, 2)))
    model.add(Convolution2D(1, (5,5), padding='same'))
    model.add(Activation('tanh'))
    return model

In [2]:
# generator_modelが(28, 28, 1)の画像を出力していることを確認
model = generator_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              103424    
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6272)              6428800   
_________________________________________________________________
batch_normalization_1 (Batch (None, 6272)              25088     
_________________________________________________________________
activation_1 (Activation)    (None, 6272)              0         
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 128)         0

In [3]:
# discriminatorを作成

from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Flatten, Dropout

def discriminator_model():
    model = Sequential()
    model.add(Convolution2D(64, (5, 5),
                            strides=(2, 2),
                            padding='same',
                            input_shape=(28, 28, 1)))
    model.add(LeakyReLU(0.2))
    model.add(Convolution2D(128, (5, 5), strides=(2, 2)))
    model.add(LeakyReLU(0.2))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [4]:
# discriminator_modelの中身、最後はsigmoidで確率を出力
model = discriminator_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 5, 128)         204928    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 5, 5, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)              

In [5]:
# 学習部分

import os
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.optimizers import Adam
from PIL import Image

BATCH_SIZE = 32
NUM_EPOCH = 20
GENERATED_IMAGE_PATH = 'generated_images/' # 生成画像の保存先

#loadした画像は0-1でクリップするのではなく、-1から1でクリップするのがGANでは一般的なようです。
#そのため、/255.0ではなく、127.5を引いてから/127.5します。
(X_train, y_train), (_, _) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2],1)
print(X_train.shape)

# discriminator
discriminator = discriminator_model()
d_opt = Adam(lr=1e-5, beta_1=0.1)
discriminator.compile(loss='binary_crossentropy', optimizer=d_opt)

# generator+discriminator （discriminator部分の重みは固定）
discriminator.trainable = False
generator = generator_model()
dcgan = Sequential([generator, discriminator])
g_opt = Adam(lr=2e-4, beta_1=0.5)
dcgan.compile(loss='binary_crossentropy', optimizer=g_opt)

# 1 epochの中のminibach数を定義
num_batches = int(X_train.shape[0] / BATCH_SIZE) # num_batches= 1875
print('Number of batches:', num_batches)
for epoch in range(NUM_EPOCH):

    # -1から1までの100次元の乱数をBATCH_SIZE(=32)個生成。これがGeneratorの入力になる。shape=(32,100)
    # データセットの方からも32枚の画像を抽出
    # generated_imagesがGeneratorによって生成された画像
    for index in range(num_batches):
        noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
        image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
        generated_images = generator.predict(noise, verbose=0)

        # 500batch学習毎に生成画像を出力
        if index % 500 == 0:

            # generate images and shape
            generated_images_plot = generated_images.astype('float32') * 127.5 + 127.5
            generated_images_plot = generated_images_plot.reshape((BATCH_SIZE, 28, 28))

            plt.figure(figsize=(8, 4))
            plt.suptitle('epoch=%04d,index=%04d' % (epoch, index), fontsize=20)
            for i in range(BATCH_SIZE):
                plt.subplot(4, 8, i + 1)
                plt.imshow(generated_images_plot[i])
                plt.gray()
                # eliminate ticks
                plt.xticks([]), plt.yticks([])


            # save images
            if not os.path.exists(GENERATED_IMAGE_PATH):
                os.mkdir(GENERATED_IMAGE_PATH)
            filename = GENERATED_IMAGE_PATH + "MNIST_%04d_%04d.png" % (epoch,index)
            plt.savefig(filename)
                
        # discriminatorを更新
        X = np.concatenate((image_batch, generated_images))
        y = [1]*BATCH_SIZE + [0]*BATCH_SIZE #教師データ:1が32個連続し、0が32個連続したベクトル
        y = np.asarray(y).astype('float32').reshape((-1,1))
        d_loss = discriminator.train_on_batch(X, y) #.train_on_batchは1つのバッチで勾配を更新

        # generatorを更新
        noise = np.array([np.random.uniform(-1, 1, 100) for _ in range(BATCH_SIZE)])
        y = [1]*BATCH_SIZE
        g_loss = dcgan.train_on_batch(noise, np.asarray(y).astype('float32').reshape((-1,1))) # モデルdcganの学習、この時descriminatorの重みは更新されない
        print("epoch: %d, batch: %d, g_loss: %f, d_loss: %f" % (epoch, index, g_loss, d_loss))

    generator.save_weights('generator.h5')
    discriminator.save_weights('discriminator.h5')


Output hidden; open in https://colab.research.google.com to view.

In [6]:
BATCH_SIZE = 32
y = [1]*BATCH_SIZE + [0]*BATCH_SIZE
y = np.asarray(y).astype('float32').reshape((-1,1))
y

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)